In [ ]:
import numpy as np
import pandas as pd

import re
import string

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LassoCV, LinearRegression, LogisticRegression, RidgeCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

from sklearn.preprocessing import MultiLabelBinarizer, Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn.metrics as metrics

import spacy
import eli5

from eli5.lime import TextExplainer

In [ ]:
df = pd.read_csv('./clean_df.csv')

In [ ]:
df.head()

In [ ]:
spacy_nlp = spacy.load('en_core_web_sm')

In [ ]:
df.head()

In [ ]:
df['spacy_tokens'] = df.job_description.apply(spacy_nlp)

In [ ]:
df_spacy = df[['spacy_tokens', 'salary']]

In [ ]:
df_spacy.head()

In [ ]:
temp = df_spacy.spacy_tokens[3]

In [ ]:
temp.ents

In [ ]:
for i in temp.noun_chunks:
    print(i)

### Manualling creating a word count vect but based of spacy's noun chunks

In [ ]:
spacy_tokens = df_spacy.spacy_tokens.apply(lambda x: [y.text for y in x.noun_chunks])

In [ ]:
biner = MultiLabelBinarizer()
chunk_counts = biner.fit_transform(spacy_tokens)

In [ ]:
feature_names = []
for i in biner.classes_:
    feature_names.append(i.replace('\n',''))

In [ ]:
feature_names[0]

In [ ]:
chonk_df = pd.DataFrame(columns=feature_names,data=chunk_counts)

In [ ]:
chonk_df.head()

In [ ]:
trimmed_features = chonk_df.sum().sort_values(ascending=False).iloc[:1000]

In [ ]:
chonk_trimmed = chonk_df[trimmed_features.index]

In [ ]:
list(trimmed_features.index)

In [ ]:
chonk_trimmed.drop(['PRIVACY POLICY.http://www.bluefinresources.com.au/privacy-policy', '|', '%', 'https://services.anu.edu.au/human-resources/respect-inclusionApplication information'],axis=1,inplace=True)

In [ ]:
chonk_trimmed.head()

In [ ]:
chonk_df.shape

In [ ]:
y = df_spacy.salary

In [ ]:
y_catog = pd.qcut(y,2,[0,1])

In [ ]:
chonk_X_train_clas, chonk_X_test_clas, chonk_y_train_clas, chonk_y_test_clas =  train_test_split(chonk_trimmed,y_catog)

In [ ]:
chonk_X_train, chonk_X_test, chonk_y_train, chonk_y_test =  train_test_split(chonk_trimmed,y)

### Modelling manual spacy noun vect with terrible results

In [ ]:
light_regres = LGBMRegressor()

In [ ]:
linreg = LinearRegression(normalize=True)

In [ ]:
linreg.fit(chonk_X_train,chonk_y_train)

In [ ]:
linreg.score(chonk_X_test,chonk_y_test)

In [ ]:
metrics.median_absolute_error(chonk_y_test,linreg.predict(chonk_X_test))

In [ ]:
ridge = RidgeCV(normalize=True,alphas=(np.linspace(1,250)))

In [ ]:
ridge.fit(chonk_X_train,chonk_y_train)

In [ ]:
ridge.score(chonk_X_test,chonk_y_test)

In [ ]:
metrics.median_absolute_error(chonk_y_test,ridge.predict(chonk_X_test))

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(chonk_X_train_clas,chonk_y_train_clas)

In [ ]:
rf.score(chonk_X_test_clas,chonk_y_test_clas)

In [ ]:
y_catog.value_counts(normalize=True)

In [ ]:
for i in temp.ents:
    print(i.text)
    
    

In [ ]:
location_dummies = pd.get_dummies(df.location,drop_first=True)

In [ ]:
location_dummies

In [ ]:
df_spacy = df_spacy.join(location_dummies)

In [ ]:
df_spacy.head()

### Trying modelling on spacy document vectors

In [ ]:
temp = df_spacy.spacy_tokens[0]

In [ ]:
temp.vector

In [ ]:
df_spacy.head()

In [ ]:
df_vect = df_spacy.spacy_tokens.apply(lambda x: pd.Series(x.vector))
# df_spacy.join()
df_vect.columns = ['vect_'+str(col) for col in df_vect.columns]

In [ ]:
df_spacy = df_spacy.join(df_vect)

In [ ]:
df_spacy.head()

In [ ]:
y = df_spacy.salary

In [ ]:
df_spacy_vect = df_spacy.drop(['spacy_tokens', 'salary'],axis=1)

In [ ]:
df_spacy_vect.head()

### Again terrible model results, starting to think there is an issue with the raw data. maybe not enough??

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_spacy_vect,y)

In [ ]:
lbm_regr = LGBMRegressor(boosting_type='gbdt',learning_rate=0.05,max_depth=3,n_estimators=1500,num_leaves=15)

In [ ]:
lbm_params = {'max_depth': [3,5,6,9,12], 'num_leaves': [15, 31, 63, 127, 255, 511, 1023, 2047, 4095], 'n_estimators': [100,500,1000,1300,1500]}

In [ ]:
grid_search = GridSearchCV(lbm_regr,lbm_params,n_jobs=-1)
grid_search.fit(X_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
lbm_regr.fit(X_train,y_train)

In [ ]:
lbm_regr.score(X_test,y_test)

In [ ]:
metrics.r2_score(lbm_regr.predict(X_test),y_test)

In [ ]:
metrics.median_absolute_error(lbm_regr.predict(X_test),y_test)

In [ ]:
ridge = RidgeCV()

In [ ]:
ridge.fit(X_train,y_train)

In [ ]:
metrics.median_absolute_error(ridge.predict(X_test),y_test)

In [ ]:
y_catog

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_spacy_vect,y_catog)

In [ ]:
lbm_clas = LGBMClassifier()

In [ ]:
lbm_clas.fit(X_train,y_train)

In [ ]:
lbm_clas.score(X_test,y_test)

In [ ]:
y_catog.value_counts(normalize=True)

In [ ]:
eli5.explain_weights(lbm_clas)

### Trying Naive Bayes on Tfidf Vector of job description

In [ ]:
t_vect = TfidfVectorizer(stop_words='english')

In [ ]:
vect = t_vect.fit_transform(df.job_description)

In [ ]:
y = pd.qcut(df.salary,3,[1,2,3])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vect,y)

In [ ]:
Mm_baybay = MultinomialNB()

In [ ]:
Mm_baybay.fit(X_train,y_train)

In [ ]:
Mm_baybay.score(X_test,y_test)

In [ ]:
Mm_baybay.classes_

In [ ]:
pipe = make_pipeline(t_vect,Normalizer(),Mm_baybay)

In [ ]:
df.job_description[0]

In [ ]:
Mm_baybay.fit(X_train,y_train)

In [ ]:
te = TextExplainer()
te.fit(df.job_description[0],pipe.predict_proba)

In [ ]:
te.show_prediction()

In [ ]:
vect.get_feature_names()

### Slightly better results. Still think something is up with raw data.

### going to import and use pre-prepared job ad data to run some models to see if there is an issue with my scrapped data

In [ ]:
df = pd.read_csv('./validation.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
type(df['job_description_all_text.1'])

In [ ]:
type(df.Bag_of_words[0])

In [ ]:
temp = df.job_description_all_text.iloc[4746]

In [ ]:
remove_salary(temp)

In [ ]:
temp.isna().sum()

In [ ]:
def remove_salary(x):
    x = re.sub('.*\$(\d+,\d).*',string=x,repl='')
    x = re.sub('.*\$(\d+).*',string=x,repl='')
    return x

In [ ]:
y = df.salary

In [ ]:

# plt

In [ ]:
y.plot(kind='density')

In [ ]:
df.job_description_all_text = df.job_description_all_text.apply(remove_salary)

In [ ]:
df.job_title = df.job_title.apply(remove_salary)

In [ ]:
df.iloc[4746]

In [ ]:
df.drop(['job_description_all_text.1' ,'Bag_of_words'],inplace=True,axis=1)

In [ ]:
df.iloc[4746,0:1]

In [ ]:
df.job_title.isna().sum()

In [ ]:
dums = pd.get_dummies(df[['company','location','industry']])

In [ ]:
df_1 = df.join(dums)

In [ ]:
df_1.head()

In [ ]:
df_1.drop(['company','location','industry','salary'],inplace=True,axis=1)

In [ ]:
df_1.head()

In [ ]:
y_binary = pd.qcut(y,2,[0,1])

In [ ]:
y_binary

In [ ]:
lbm_clasif = LGBMClassifier(objective='binary',learning_rate=0.05)

In [ ]:
t_vect = TfidfVectorizer(max_df=300,max_features=1000,min_df=10,ngram_range=(1,3))
# Params for job_description - max_df=150,max_features=1000,min_df=10,ngram_range=(1,3)
# Params for job_title - max_df=300,max_features=1000,min_df=10,ngram_range=(1,3)

In [ ]:
pipe = make_pipeline(t_vect,Normalizer(),lbm_clasif)

In [ ]:
pipe.get_params().keys()

In [ ]:
params = {'tfidfvectorizer__ngram_range':[(1,3),(1,6)],'tfidfvectorizer__max_df':[300,500,1000],'tfidfvectorizer__max_features': [1000],'tfidfvectorizer__min_df':[10],
         }

In [ ]:
grid_pipe = GridSearchCV(pipe, params,verbose=1,n_jobs=-1)

In [ ]:
grid_pipe.fit(df_1.job_description_all_text,y_binary)

In [ ]:
grid_pipe.best_params_

In [ ]:
job_descr_tvec = t_vect.fit_transform(df_1.job_description_all_text)

In [ ]:
job_descr_tvec

In [ ]:
grid_pipe.fit(df_1.job_title,y_binary)

In [ ]:
grid_pipe.best_params_

In [ ]:
job_title_tvec = t_vect.fit_transform(df_1.job_title)

In [ ]:
job_des_df = pd.DataFrame(job_descr_tvec.todense(), columns=t_vect.get_feature_names())

In [ ]:
job_des_df.head()

In [ ]:
job_title_df = pd.DataFrame(job_title_tvec.todense(), columns=t_vect.get_feature_names())

In [ ]:
df_1.head()

In [ ]:
df_2 = df_1.drop(['job_description_all_text', 'job_title'],axis=1)

In [ ]:
df_2.head()

In [ ]:
df_2 = df_2.join(job_des_df)

In [ ]:
df_2.head()

In [ ]:
df_2.isna().sum()

In [ ]:
rating_avg = np.mean(df.company_rating)

In [ ]:
df_2.company_rating.fillna(rating_avg,inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_2,y_binary)

In [ ]:
clasif_pipe = make_pipeline(Normalizer(),LGBMClassifier(max_depth=6,n_estimators=500,num_leaves=127,learning_rate=0.05,n_jobs=-1))

In [ ]:
clasif_params = {'lgbmclassifier__max_depth': [5,7,12], 'lgbmclassifier__num_leaves': [255, 511, 1023, 2047], 'lgbmclassifier__n_estimators': [100,500,1000,1500]}

In [ ]:
classif_grid = GridSearchCV(clasif_pipe,clasif_params,verbose=2,n_jobs=-1)

In [ ]:
classif_grid.fit(X_train,y_train)

In [ ]:
classif_grid.best_params_

In [ ]:
clasif_pipe.fit(X_train.values,y_train.values)

In [ ]:
clasif_pipe.score(X_test.values,y_test.values)

In [ ]:
y_binary.value_counts(normalize=True)

In [ ]:
import eli5

In [ ]:
eli5.explain_weights(clasif_pipe[-1],feature_names=df_2.columns.tolist())

In [ ]:
df_2.columns